In [2]:
import pandas as pd
import datetime

%store -r query_vars

In [3]:
query_vars

{'wikis_to_query': ('wikishared',),
 'snapshot': '2023-09',
 'metrics_month': '2023-09',
 'metrics_month_start': '2023-09-01 00:00:00',
 'metrics_month_first_day': '2023-09-01',
 'metrics_month_end': '2023-10-01 00:00:00',
 'year': '2023',
 'month': '09',
 'month_int': 9,
 'month_int_end': 10,
 'utc_start': '20230901000000',
 'utc_end': '20231001000000',
 'day_before_utc_end': '20230801000000',
 'metrics_month_last_day': '2023-09-30',
 'api_metrics_month_first_day': '20230901',
 'api_metrics_month_next_month_first_day': '20231001',
 'api_metrics_month_day_after': '20231001',
 'metrics_prev_month': '2023-08',
 'retention_cohort': '2023-07'}

In [ ]:
event_data  = pd.read_csv('output/wrangling/base_view.csv')  

parse_dates_editors = ['cep_registered_at', 'cep_unregistered_at','last_activity_date','ac_date']#'gu_registration'

editor_data = pd.read_csv('output/wrangling/editors.csv',parse_dates=parse_dates_editors)

In [ ]:
# datetime
event_data['event_created_at'] = pd.to_datetime(event_data['event_created_at'], format = "%Y%m%d%H%M%S")#.dt.strftime("%Y%m%d%H")
event_data['event_deleted_at'] = pd.to_datetime(event_data['event_deleted_at'], format = "%Y%m%d%H%M%S")#.dt.strftime("%Y%m%d%H")
event_data['event_last_edit'] = pd.to_datetime(event_data['event_last_edit'], format = "%Y%m%d%H%M%S")#.dt.strftime("%Y%m%d%H")
event_data['event_start_utc'] = pd.to_datetime(event_data['event_start_utc'], format = "%Y%m%d%H%M%S")#.dt.strftime("%Y%m%d%H")
event_data['event_end_utc'] = pd.to_datetime(event_data['event_end_utc'], format = "%Y%m%d%H%M%S")#.dt.strftime("%Y%m%d%H")

In [5]:
pd.set_option('display.max_columns', None)

## handle event type/status
### address event dates and whether events are active or registers or tests, etc.

| `active_event` column category | description |
|:----------|:--------|
| `create` | events that will happen in the future that were created in this period |
| `test_register` | event registers, on test events, in this period  |
| `test_edit` | organizer edited the event in this period and the event is a test event |
| `deleted` | event deletes in this period |
| `future_event_editing` | organizer edited the event in this period, the event takes place in the future |
| `inprogress_single_month` | active events happening in this period only |
| `inprogress_multi_month` | started this period, ongoing |
| `inprogress_multi_month` | started previous period, ending this period |
| `inprogress_multi_month` | started previous period, ongoing |


In [6]:
#for categorizing as 'inprogress_single_month'
event_active_mask = ((event_data['event_start_utc'] >= query_vars['utc_start']) & 
                        (event_data['event_end_utc'] < query_vars['utc_end'])
                       )

# for categorizing as 'create'
event_register_mask = ((event_data['event_created_at'] >= query_vars['utc_start']) & 
                       (event_data['event_created_at'] < query_vars['utc_end']) 
                       #& (event_data['event_start_utc']>= query_vars['utc_end'])
                      )


event_last_edit_mask = ((event_data['event_last_edit'] >= query_vars['utc_start']) & 
                       (event_data['event_last_edit'] < query_vars['utc_end']))

event_deleted_mask = ((event_data['event_deleted_at'] >= query_vars['utc_start']) &
                      (event_data['event_deleted_at'] < query_vars['utc_end']))

In [7]:
#register_test: event registers in the last month that are test events
event_data.loc[(event_register_mask) &
               (event_data['event_deleted_at'].isna()) &
               (event_data['event_status'] == 5), 'active_event'] = 'test_register'

In [8]:
event_data[['event_id', 'event_status','active_event','organizer_username','event_deleted_at','event_start_utc', 'event_end_utc','event_created_at','event_last_edit']][event_data['active_event']=='register_test']

,event_id,event_status,active_event,organizer_username,event_deleted_at,event_start_utc,event_end_utc,event_created_at,event_last_edit


In [9]:
#edit_test: event edits in the last month on test events
event_data.loc[(event_last_edit_mask) &
               (event_data['event_deleted_at'].isna()) &
               (event_data['event_status'] == 5), 'active_event'] = 'test_edit'

In [10]:
#'inprogress_single_month': active events happening in this period only 
event_data.loc[(event_active_mask) &
               (event_data['event_status'] != 5) &
               (event_data['event_deleted_at'].isna()), 'active_event'] = 'inprogress_single_month'

In [12]:
#create: events that will happen in the future that were created in this period
event_data.loc[(event_register_mask) & 
               (event_data['event_status'] != 5) &
               (event_data['event_deleted_at'].isna()), 'active_event'] = 'create'

In [14]:
#inprogress_multi_month: started this period, ongoing
event_data.loc[(event_data['active_event'].isna()) & 
               (event_data['event_start_utc'].dt.month==query_vars['month_int']) & 
               (event_data['event_status'] != 5) &
               (event_data['event_end_utc'] > query_vars['utc_end']), 'active_event'] = 'inprogress_multi_month'

In [15]:
#inprogress_multi_month: started previous period, ongoing
event_data.loc[(event_data['active_event'].isna()) & 
               (event_data['event_start_utc']<= query_vars['utc_start']) & 
               (event_data['event_status'] != 5) &
               (event_data['event_end_utc'] > query_vars['utc_end']), 'active_event'] = 'inprogress_multi_month'

In [16]:
#inprogress_multi_month: started previous period, ending this period
event_data.loc[(event_data['active_event'].isna()) & 
               (event_data['event_status'] != 5) &
               (event_data['event_end_utc'].dt.month==query_vars['month_int']), 'active_event'] = 'inprogress_multi_month'

In [17]:
#future_event_editing: organizer edited the event in this period, the event takes place in the future
event_data.loc[(event_data['active_event'].isna()) & 
               (event_data['event_status'] != 5) &
               (event_data['event_last_edit'].dt.month==query_vars['month_int']), 'active_event'] = 'future_event_editing'

In [18]:
#event deletes in this period
event_data.loc[event_deleted_mask, 'active_event'] = 'deleted'

In [19]:
event_data[['event_id', 'event_status','event_deleted_at','organizer_username','event_start_utc', 'event_end_utc','event_created_at','event_last_edit']][event_data['active_event']=='deleted']

,event_id,event_status,event_deleted_at,organizer_username,event_start_utc,event_end_utc,event_created_at,event_last_edit


In [20]:
z = (event_data[['event_id', 'event_status','event_deleted_at','organizer_username','event_start_utc', 
            'event_end_utc','event_created_at','event_last_edit']][event_data['active_event'].isna()])

In [21]:
z[z['event_status']==5]

,event_id,event_status,event_deleted_at,organizer_username,event_start_utc,event_end_utc,event_created_at,event_last_edit


In [22]:
#reorder columns
event_data = event_data[['event_id','event_status','active_event','event_page_title',
                                   'country','wmf_region', 
                                   'pageviews',
                                   'organizers','organizer_username',
                                   'new_users_count',
                                   'participants',
                                   'event_created_at','event_start_utc', 'event_end_utc', 'event_last_edit','event_deleted_at',
                                   'event_chat_url_clean','cett_tool_id','cett_tool_event_id','event_meeting_url',
                                   'participants_register_count','participant_unregister_count',
                                   'event_type','event_meeting_type'
                                  ]]

In [ ]:
## remove events created internally for testing by organizer username

In [23]:
#TODO - For meta, all test events should be subpages of Event:Sandbox. If that's not the case, then those that don't comply should be moved.
#see conversation on slack: https://wikimedia.slack.com/archives/C020V0GN9P1/p1690976380675029?thread_ts=1690924097.149909&cid=C020V0GN9P1
#remove tester data
#list, discussion: https://wikimedia.slack.com/archives/C020V0GN9P1/p1690924134855739?thread_ts=1690924097.149909&cid=C020V0GN9P1
do_not_include_list = ['VWalters-WMF', 
                       'QTE-CampaignEvents0', 'QTE-CampaignEvents1', 
                       'QTE-CampaignEvents2', 'QTE-CampaignEvents3', 'QTE-CampaignEvents4', 
                       'QTE-CampaignEvents5', 'QTE-CampaignEvents6', 'QTE-CampaignEvents7','LDelench (WMF)']


df_exclude = pd.DataFrame({'testers': do_not_include_list})


for a in event_data.organizer_username.index:
    for search in df_exclude.testers:
        if search in event_data.loc[a].organizer_username:
            event_data.drop(a, inplace=True)
            break  

In [7]:
event_data[~event_data['event_deleted_at'].isna()]

,event_id,event_status,active_event,event_page_title,country,wmf_region,pageviews,organizers,organizer_username,new_users_count,...,event_last_edit,event_deleted_at,event_chat_url_clean,cett_tool_id,cett_tool_event_id,event_meeting_url,participants_register_count,participant_unregister_count,event_type,event_meeting_type


In [24]:
event_data.to_csv('output/handling/metrics.csv', index=False)

## handle editor data

In [38]:
editor_data_df = editor_data.merge(event_data[['event_id', 'event_type']], on = 'event_id', how='left')

In [39]:
editor_data_df.to_csv('output/handling/editors.csv', index=False)

In [ ]:
#remove tester data